# Проектная работа: анализ посещаемости магазинов с помощью нейронных сетей в компьютерном зрении

## 1. Импорт зависимостей

In [ ]:
import json
import random
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

import cv2 as cv
import torch
from ultralytics import YOLO

## 2. Общие конфигурации и настройки

In [ ]:
warnings.filterwarnings("ignore")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [ ]:
FORCE_CPU = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 3. Подготовка данных


### 3.1. Конфигурация и схема данных

In [ ]:
CUR_DIR = Path.cwd()
PROJECT_DIR = CUR_DIR.parent

DATA_DIR = PROJECT_DIR / "data"
MODEL_DIR = PROJECT_DIR / "models"

for dir in [DATA_DIR, MODEL_DIR]:
    dir.mkdir(parents=True, exist_ok=True)

In [ ]:
VIDEOS_SHAPE = (720, 720)
VIDEOS_FPS = 29.0

MODEL_INPUT_SHAPE = (640, 640)

In [ ]:
class Shop:
    def __init__(self, name, pts):
        self.name = name
        self.pts = np.array(pts)
        self.visit_count = 0

    def __repr__(self):
        return f"Shop {self.name} (pts={self.pts})"


def draw_shop(img, shop):
    cv.polylines(img, [shop.pts], isClosed=True, color=(0, 255, 0), thickness=3)

In [ ]:
input_videos_filenames = [
    DATA_DIR / "input_1.mp4",
    DATA_DIR / "input_2.mp4",
    DATA_DIR / "input_3.mp4",
    DATA_DIR / "input_4.mp4",
]

In [ ]:
SHOPS_CONFIG_FILE = DATA_DIR / "shops_config.json"

### 3.2. Экземпляры данных

In [ ]:
with open(SHOPS_CONFIG_FILE, "r") as file:
    shops_configs = json.load(file)

shops = [Shop(**shop_cfg) for shop_cfg in shops_configs]

In [ ]:
# Testing the functionality and correctness of the configuration

ex_idx = 0

input_video_filename = input_videos_filenames[ex_idx]

cap = cv.VideoCapture(str(input_video_filename))

if not cap.isOpened():
    print("Error opening video file")
else:
    ret, frame = cap.read()

    for shop in shops:
        draw_shop(frame, shop)
    cv.imshow("Video Frame", frame)

    cv.waitKey(0)
    cap.release()
    cv.destroyAllWindows()

## 4. Нейросетевая модель

In [ ]:
MODEL_FILE = MODEL_DIR / "yolo11s.pt"

In [ ]:
class PeopleDetectionYOLO:
    def __init__(self, model_file=MODEL_FILE, device=FORCE_CPU):
        self.model = YOLO(model_file).to(device).eval()

    def predict(self, img_tensor):
        with torch.no_grad():
            return self.model(img_tensor)

In [ ]:
model = PeopleDetectionYOLO(device=device)